# MIDI preprocessing

In [1]:
import mido
import numpy as np
import time
import copy

### Read MIDI file
* 기존의 미디 파일을 읽어온다.
* tick per beat를 확인할 수 있다.

In [2]:
mid = mido.MidiFile('school.mid')
print(mid.ticks_per_beat)

96


### MIDI 메세지를 불러오는 방법에 대한 차이
* 파일 자체에서 불러올 시 tick이 ms로 계산되어 표시
* track 에서 불러올 시 tick으로 표시

In [3]:
for msg in mid:
    print(msg)
  

<meta message track_name name='\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=0.25
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=0.25
note_on channel=0 note=69 velocity=100 time=0
note_off channel=0 note=69 velocity=64 time=0.25
note_on channel=0 note=69 velocity=100 time=0
note_on channel=0 note=67 velocity=100 time=0.25
note_off channel=0 note=69 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=0.25
note_on channel=0 note=67 velocity=100 time=0
note_on channel=0 note=64 velocity=100 time=0.25
note_off channel=0 note=67 velocity=64 time=0
note_off channel=0 note=64 velocity=64 time=0.5
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 

In [4]:
# beat resolution == 192
for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)

Track 0:  
<meta message track_name name='\x00' time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=36 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=69 velocity=100 time=0
note_off channel=0 note=69 velocity=64 time=48
note_on channel=0 note=69 velocity=100 time=0
note_on channel=0 note=67 velocity=100 time=48
note_off channel=0 note=69 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=67 velocity=100 time=0
note_on channel=0 note=64 velocity=100 time=48
note_off channel=0 note=67 velocity=64 time=0
note_off channel=0 note=64 velocity=64 time=96
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 no

### MIDI 데이터 파싱
* 데이터에서 중요한 ON OFF 데이터 파싱
* Velocity == 0 인 경우도 파악할 것

In [5]:
#event : [Time, Type(ON, OFF, CC), Value1, Value2]
def get_eventlist(data_file):
    ON = 1
    OFF = 0

    midi = mido.MidiFile(data_file)

    current_time = 0
    eventlist = []
    
    for msg in midi:
#         current_time += msg.time
        current_time = msg.time
        
         # NOTE ON CASE
        if msg.type is 'note_on' and msg.velocity > 0:
            event = [current_time, ON, msg.note, msg.velocity]
            eventlist.append(event)

         # NOTE OFF CASE        
        elif msg.type is 'note_off' or (msg.type is 'note_on' and msg.velocity == 0):
            event = [current_time, OFF, msg.note, msg.velocity]
            eventlist.append(event)
                
    return eventlist

### 미디 데이터 수정
* 같은 길이 만큼 뒤로 복사

In [6]:
midi_data = get_eventlist('school.mid')
print(midi_data)

[[0, 1, 67, 100], [0.25, 0, 67, 64], [0, 1, 67, 100], [0.25, 0, 67, 64], [0, 1, 69, 100], [0.25, 0, 69, 64], [0, 1, 69, 100], [0.25, 1, 67, 100], [0, 0, 69, 64], [0.25, 0, 67, 64], [0, 1, 67, 100], [0.25, 1, 64, 100], [0, 0, 67, 64], [0.5, 0, 64, 64], [0, 1, 67, 100], [0.25, 0, 67, 64], [0, 1, 67, 100], [0.25, 1, 64, 100], [0, 0, 67, 64], [0.25, 0, 64, 64], [0, 1, 64, 100], [0.25, 1, 62, 100], [0, 0, 64, 64], [0.5, 0, 62, 64]]


In [7]:
dup = copy.deepcopy(midi_data)
print(dup)

[[0, 1, 67, 100], [0.25, 0, 67, 64], [0, 1, 67, 100], [0.25, 0, 67, 64], [0, 1, 69, 100], [0.25, 0, 69, 64], [0, 1, 69, 100], [0.25, 1, 67, 100], [0, 0, 69, 64], [0.25, 0, 67, 64], [0, 1, 67, 100], [0.25, 1, 64, 100], [0, 0, 67, 64], [0.5, 0, 64, 64], [0, 1, 67, 100], [0.25, 0, 67, 64], [0, 1, 67, 100], [0.25, 1, 64, 100], [0, 0, 67, 64], [0.25, 0, 64, 64], [0, 1, 64, 100], [0.25, 1, 62, 100], [0, 0, 64, 64], [0.5, 0, 62, 64]]


In [8]:
for data in dup:
    midi_data.append(data)

In [9]:
get_dup_midi = np.array(midi_data)

In [10]:
get_dup_midi

array([[  0.  ,   1.  ,  67.  , 100.  ],
       [  0.25,   0.  ,  67.  ,  64.  ],
       [  0.  ,   1.  ,  67.  , 100.  ],
       [  0.25,   0.  ,  67.  ,  64.  ],
       [  0.  ,   1.  ,  69.  , 100.  ],
       [  0.25,   0.  ,  69.  ,  64.  ],
       [  0.  ,   1.  ,  69.  , 100.  ],
       [  0.25,   1.  ,  67.  , 100.  ],
       [  0.  ,   0.  ,  69.  ,  64.  ],
       [  0.25,   0.  ,  67.  ,  64.  ],
       [  0.  ,   1.  ,  67.  , 100.  ],
       [  0.25,   1.  ,  64.  , 100.  ],
       [  0.  ,   0.  ,  67.  ,  64.  ],
       [  0.5 ,   0.  ,  64.  ,  64.  ],
       [  0.  ,   1.  ,  67.  , 100.  ],
       [  0.25,   0.  ,  67.  ,  64.  ],
       [  0.  ,   1.  ,  67.  , 100.  ],
       [  0.25,   1.  ,  64.  , 100.  ],
       [  0.  ,   0.  ,  67.  ,  64.  ],
       [  0.25,   0.  ,  64.  ,  64.  ],
       [  0.  ,   1.  ,  64.  , 100.  ],
       [  0.25,   1.  ,  62.  , 100.  ],
       [  0.  ,   0.  ,  64.  ,  64.  ],
       [  0.5 ,   0.  ,  62.  ,  64.  ],
       [  0.  , 

### 복사된 데이터로 미디파일 생성

In [11]:
from mido import Message, MidiFile, MidiTrack, MetaMessage, bpm2tempo

mid = MidiFile()
mid.ticks_per_beat = 96

track = MidiTrack()
mid.tracks.append(track)
track.append(MetaMessage("set_tempo", tempo=bpm2tempo(120)))
track.append(MetaMessage("time_signature", numerator=4, denominator=4, clocks_per_click=36))

for data in get_dup_midi:
    type = {0:"note_off", 1:"note_on"}
    data_type = type[data[1]]
    track.append(Message(data_type, note=int(data[2]), velocity=int(data[3]), time=int(data[0]*192)))

track.append(MetaMessage("end_of_track"))

for i, track in enumerate(mid.tracks):
    print('Track {}: {}'.format(i, track.name))
    for msg in track:
        print(msg)
        
mid.save("new_school.mid")

Track 0: 
<meta message set_tempo tempo=500000 time=0>
<meta message time_signature numerator=4 denominator=4 clocks_per_click=8 notated_32nd_notes_per_beat=8 time=0>
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=69 velocity=100 time=0
note_off channel=0 note=69 velocity=64 time=48
note_on channel=0 note=69 velocity=100 time=0
note_on channel=0 note=67 velocity=100 time=48
note_off channel=0 note=69 velocity=64 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=67 velocity=100 time=0
note_on channel=0 note=64 velocity=100 time=48
note_off channel=0 note=67 velocity=64 time=0
note_off channel=0 note=64 velocity=64 time=96
note_on channel=0 note=67 velocity=100 time=0
note_off channel=0 note=67 velocity=64 time=48
note_on channel=0 note=67 velocity=100 time=0
note_on channel=0 note=64 velocity=100 time

### Music 21

In [4]:
import music21
from music21 import stream, midi
from music21 import converter

In [5]:
mf = midi.MidiFile()
mf.open("new_school.mid")
mf.read()
mf.close()
s = midi.translate.midiFileToStream(mf)
s.show('text')

{0.0} <music21.stream.Part 0x1136ae9d0>
    {0.0} <music21.tempo.MetronomeMark animato Quarter=120.0>
    {0.0} <music21.meter.TimeSignature 4/4>
    {0.0} <music21.note.Note G>
    {0.5} <music21.note.Note G>
    {1.0} <music21.note.Note A>
    {1.5} <music21.note.Note A>
    {2.0} <music21.note.Note G>
    {2.5} <music21.note.Note G>
    {3.0} <music21.note.Note E>
    {4.0} <music21.note.Note G>
    {4.5} <music21.note.Note G>
    {5.0} <music21.note.Note E>
    {5.5} <music21.note.Note E>
    {6.0} <music21.note.Note D>
    {7.0} <music21.note.Note G>
    {7.5} <music21.note.Note G>
    {8.0} <music21.note.Note A>
    {8.5} <music21.note.Note A>
    {9.0} <music21.note.Note G>
    {9.5} <music21.note.Note G>
    {10.0} <music21.note.Note E>
    {11.0} <music21.note.Note G>
    {11.5} <music21.note.Note G>
    {12.0} <music21.note.Note E>
    {12.5} <music21.note.Note E>
    {13.0} <music21.note.Note D>


In [6]:
for part in s.parts:
    for n in part.flat.notesAndRests:
        if n.isRest:
            dur = n.duration.quarterLength
            note_num = "<REST>"
            print(dur, note_num)
        else:
            dur = n.duration.quarterLength
            note_num = sorted(set([p.midi for p in n.pitches]))
            print(dur, list(note_num))

0.5 [67]
0.5 [67]
0.5 [69]
0.5 [69]
0.5 [67]
0.5 [67]
1.0 [64]
0.5 [67]
0.5 [67]
0.5 [64]
0.5 [64]
1.0 [62]
0.5 [67]
0.5 [67]
0.5 [69]
0.5 [69]
0.5 [67]
0.5 [67]
1.0 [64]
0.5 [67]
0.5 [67]
0.5 [64]
0.5 [64]
1.0 [62]


In [7]:
s.write('midi','new_school_21.mid')

'new_school_21.mid'